In [145]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv("order_brush_order.csv")
df

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [20]:
# make event_time calculatable
df['event_time'] = df.event_time.apply(lambda x : pd.Timestamp(x))
df

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [21]:
# sort to see how long the dataset recorded 
df.event_time.sort_values()

150060   2019-12-27 00:00:00
25985    2019-12-27 00:00:00
81020    2019-12-27 00:00:00
146598   2019-12-27 00:00:02
80355    2019-12-27 00:00:05
                 ...        
114113   2019-12-31 23:59:43
206585   2019-12-31 23:59:47
126212   2019-12-31 23:59:51
160657   2019-12-31 23:59:51
179881   2019-12-31 23:59:56
Name: event_time, Length: 222750, dtype: datetime64[ns]

In [136]:
# drop non-suspicious shops
df_groupby_shop = df.groupby("shopid")
countdf = df_groupby_shop.count()
nice_shops = countdf[countdf["orderid"] < 3].index.to_list();
new_df = df.drop(df[df["shopid"].isin(nice_shops)].index).drop("orderid", axis = 1)
new_df.sort_values("shopid")

,shopid,userid,event_time
200439,10061,130633421,2019-12-30 21:35:15
145426,10061,62464559,2019-12-31 02:58:48
188513,10061,168750452,2019-12-28 09:27:55
174620,10061,194819216,2019-12-28 12:05:32
109999,10084,136264815,2019-12-30 21:14:03
...,...,...,...
110351,213900783,124465026,2019-12-29 22:17:00
83893,214432425,52594422,2019-12-31 23:45:49
133194,214432425,1134243,2019-12-31 23:32:03
4590,214432425,52594422,2019-12-31 23:46:35


In [164]:
f = df_groupby_shop.get_group(133747795).event_time.iloc[0]
l = df_groupby_shop.get_group(133747795).event_time.iloc[-1]

In [171]:
(l-f).total_seconds()

221165.0

In [105]:
# test dict
test_df = pd.DataFrame([], columns = ["shopid", "userid"])
test_dict = {1000: [300, 833, 910], 1403: [6393, 333, 787], 1555: [877, 933, 103]}
for s in test_dict.keys():
    test_df = test_df.append({'shopid':s, 'userid':"&".join(str(x) for x in test_dict[s])}, ignore_index = True)
    
test_df

,shopid,userid
0,1000,300&833&910
1,1403,6393&333&787
2,1555,877&933&103


In [190]:
suspicious_df = pd.DataFrame([], columns = ["shopid", "userid"])
monster_shops = {}
suspicious_dict = {}
for shopid in df_groupby_shop.groups.keys():
    shopdata = df_groupby_shop.get_group(shopid)
    time_series = shopdata.sort_values("event_time")["event_time"]
    
    tslen = len(time_series)
    if tslen >= 400:
        print("The shop[id={}] has a large order number: {}".format(shopid, tslen))
        monster_shops[shopid] = tslen
#         suspicious_dict[shopid] = [0]
#         continue
    for i in range(tslen):
        num_of_orders = 1
        users_set = [shopdata.iloc[i].userid]
        for j in range(i+1, tslen):
            if abs((time_series.iloc[i] - time_series.iloc[j]).total_seconds()) <= 3600:
                num_of_orders += 1
                uid = shopdata.iloc[j].userid
                if uid not in users_set:
                    users_set.append(uid)
            else:
                break

        concentrate = num_of_orders / len(users_set)
        if shopid in suspicious_dict.keys():
            if concentrate >= 3:
                if 0 in suspicious_dict[shopid]:
                    suspicious_dict[shopid] = users_set
                else:
                    new_users = set(users_set) - set(suspicious_dict[shopid])
                    suspicious_dict[shopid].extend(list(new_users))
        else:
            if concentrate >= 3:
                suspicious_dict[shopid] = users_set
            else:
                suspicious_dict[shopid] = [0]
                
for s in suspicious_dict.keys():
    suspicious_df  = suspicious_df.append({'shopid':s, 'userid':"&".join(str(x) for x in suspicious_dict[s])}, ignore_index = True)

suspicious_df

The shop[id=21672386] has a large order number: 446
The shop[id=24013303] has a large order number: 437
The shop[id=25924280] has a large order number: 655
The shop[id=26974701] has a large order number: 430
The shop[id=32504026] has a large order number: 516
The shop[id=34910509] has a large order number: 515
The shop[id=43412276] has a large order number: 529
The shop[id=54615708] has a large order number: 2640
The shop[id=57189823] has a large order number: 687
The shop[id=61556313] has a large order number: 6691
The shop[id=64909117] has a large order number: 481
The shop[id=84301799] has a large order number: 501
The shop[id=89768730] has a large order number: 607
The shop[id=91799978] has a large order number: 707
The shop[id=107921853] has a large order number: 684
The shop[id=147941492] has a large order number: 11703
The shop[id=156498720] has a large order number: 511


,shopid,userid
0,10009,0
1,10051,0
2,10061,0
3,10084,0
4,10100,0
...,...,...
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0


,shopid,userid
18765,214662358,0
18766,214949521,0
18767,214964814,0
18768,215175775,0
18769,215435223,0


In [191]:
view = suspicious_df.drop(suspicious_df[suspicious_df["userid"] == '0'].index)
view

,shopid,userid
57,10536,672345
251,425364,72914921
314,731606,75558350
599,3124091,214568881
1102,10199219,8405753
2798,42023512,163101092
3133,50682734,214365114
4140,68609828,183440131&174449062&148694289
5018,83234229,181942741
6086,100446829,2434757


,orderid,shopid,userid,event_time
55453,31080716728347,133747795,147853291,2019-12-27 01:31:56
50991,31165070690262,133747795,147853291,2019-12-28 00:57:50
29342,31166210230085,133747795,147853291,2019-12-28 01:16:50
88310,31387374191083,133747795,1008857,2019-12-30 14:42:55


In [192]:
suspicious_df.to_csv('Submission.csv', index=False)

In [84]:
resultdf = pd.read_csv("Order.csv")
new_resultdf = resultdf.drop(resultdf[resultdf["userid"] == '0'].index)
new_resultdf

,shopid,userid
61,100856387,1033878&1033878
150,102200925,11356448&11356448
301,10446,8767052&8767052
336,104655070,113772750&113772750
438,10556,113939321&113939321
...,...,...
18590,97371388,150543581&150543581
18604,97868707,83779951&83779951
18714,98929951,212051007&212051007
18751,99780139,207839983&207839983


In [85]:
new_resultdf.query("shopid == 68609828")

,shopid,userid
16500,68609828,183440131&183440131&183440131&183440131&183440...


In [33]:
new_df[new_df["shopid"] == 100856387].sort_values("event_time")

,shopid,userid,event_time
200855,100856387,11380217,2019-12-27 03:56:58
212898,100856387,95194629,2019-12-28 02:38:25
9417,100856387,167378981,2019-12-28 12:22:49
32374,100856387,103353177,2019-12-28 21:48:28
44397,100856387,2360427,2019-12-30 11:02:58
119043,100856387,1033878,2019-12-30 20:12:22
222066,100856387,1033878,2019-12-30 21:02:23
667,100856387,6548822,2019-12-31 09:56:56
216088,100856387,49277788,2019-12-31 23:16:54
191198,100856387,145343618,2019-12-31 23:25:54


In [83]:
new_df.query("shopid == 100856387 and userid == 1033878")

,shopid,userid,event_time
14223,68609828,15865730,2019-12-27 08:47:40
21409,68609828,183440131,2019-12-29 18:21:47
45404,68609828,15760481,2019-12-29 16:56:52
70500,68609828,183440131,2019-12-29 18:01:52
85834,68609828,183440131,2019-12-30 17:32:26
86835,68609828,183440131,2019-12-27 21:50:25
86924,68609828,183440131,2019-12-30 17:30:58
91348,68609828,183440131,2019-12-27 19:30:51
93991,68609828,174449062,2019-12-29 00:30:42
95765,68609828,183440131,2019-12-30 17:49:09
